# Data Collection

In [ ]:
%pip install requests beautifulsoup4 fake_useragent pandas openpyxl

## Web Scraping URLs

In [ ]:
import requests
from bs4 import BeautifulSoup
from fake_useragent import UserAgent
import urllib.parse

def google_search(query, num_results, time_filter = None):
    ua = UserAgent()
    headers = {'User-Agent': ua.random}

    query = urllib.parse.quote_plus(query)

    google_url = f"https://www.google.com/search?q={query}&num={num_results}"

    if time_filter:
        google_url += f"&tbs={time_filter}"

    response = requests.get(google_url, headers=headers)

    if response.status_code == 200:
        soup = BeautifulSoup(response.text, 'html.parser')
        search_results = []

        for g in soup.find_all('div', class_='g'):
            anchors = g.find_all('a')
            if anchors:
                link = anchors[0]['href']
                search_results.append(link)
                
        return search_results
    else:
        print(f"failed to retrieve search results: status code {response.status_code}")
        return []


In [ ]:
def generate_quarters(start_year, end_year):
    quarters = {}
    if end_year == 2024:
        quarters["2024 Q1"] = "cdr:1,cd_min:1/1/2024,cd_max:3/31/2024"
        quarters["2024 Q2"] = "cdr:1,cd_min:4/1/2024,cd_max:6/30/2024"
        end_year -= 1
    for year in range(start_year, end_year + 1):
        quarters[f"{year} Q1"] = f"cdr:1,cd_min:1/1/{year},cd_max:3/31/{year}"
        quarters[f"{year} Q2"] = f"cdr:1,cd_min:4/1/{year},cd_max:6/30/{year}"
        quarters[f"{year} Q3"] = f"cdr:1,cd_min:7/1/{year},cd_max:9/30/{year}"
        quarters[f"{year} Q4"] = f"cdr:1,cd_min:10/1/{year},cd_max:12/31/{year}"
    return quarters

def generate_query(source_list):
    dictionary = {}
    for source in source_list:
        if source in dictionary:
            continue
        else:
            dictionary[source] = f"singapore industrial property market {source}"
    return dictionary

In [ ]:
import pandas as pd

query_dictionary = generate_query(["cna", "singapore business review"])

quarter_dictionary = generate_quarters(2020, 2024)

headers = ["URLs", "Source", "Quarter"]
df = pd.DataFrame(columns=headers)

for source, query in query_dictionary.items():
    for quarter, time_filter in quarter_dictionary.items():
        results = google_search(query, num_results=20, time_filter=time_filter)
        temp_df = pd.DataFrame({"URLs": results, "Source": source, "Quarter": quarter})
        df = pd.concat([df, temp_df], ignore_index=True)

print(df)

query_dictionary = generate_query(["straits times", "business times", "edgeprop"])

quarter_dictionary = generate_quarters(2020, 2024)

for source, query in query_dictionary.items():
    for quarter, time_filter in quarter_dictionary.items():
        results = google_search(query, num_results=30, time_filter=time_filter)
        temp_df = pd.DataFrame({"URLs": results, "Source": source, "Quarter": quarter})
        df = pd.concat([df, temp_df], ignore_index=True)

print(df)

df.to_excel("urls.xlsx", index=False)

## URLs Cleaning

In [ ]:
df = pd.read_excel("urls.xlsx")
duplicates = df["URLs"].duplicated(keep=False)
df = df[~duplicates]

with pd.ExcelWriter("urls.xlsx", engine="openpyxl", mode="a", if_sheet_exists="replace") as writer:
    df.to_excel(writer, sheet_name="2020 - 2024", index=False)

## Date Scraping URLs

In [ ]:
import requests
from bs4 import BeautifulSoup
import pandas as pd

def get_article_date(url):
    headers = {
        'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
    }
    
    try:
        response = requests.get(url, headers=headers)
        response.raise_for_status()
    except requests.exceptions.RequestException as e:
        return f"failed to retrieve the webpage: {e}"
    
    soup = BeautifulSoup(response.content, 'html.parser')
    
    # locate the date in the meta tag with property 'article:published_time'
    date = soup.find('meta', attrs={'property': 'article:published_time'})
    if date:
        return date['content']
    
    # locate the date in the meta tag with name 'cXenseParse:recs:publishtime'
    date = soup.find('meta', attrs={'name': 'cXenseParse:recs:publishtime'})
    if date:
        return date['content']
    
    # locate the date in the meta tag with name 'article:published_time'
    date = soup.find('meta', attrs={'name': 'article:published_time'})
    if date:
        return date['content']
    
    # locate the date 
    time_element = soup.find('time')
    if time_element and 'datetime' in time_element.attrs:
        return time_element['datetime']
    
    # if not found
    return "date not found"

In [ ]:
excel_file = 'urls.xlsx'
sheet_name = '2020 - 2024'
df_existing = pd.read_excel(excel_file, sheet_name=sheet_name)

urls = df_existing['URLs'].tolist()

date_data = []
for url in urls:
    date = get_article_date(url)
    print(date)
    date_data.append({'URLs': url, 'Date': date})

df_date = pd.DataFrame(date_data)

df_updated = pd.merge(df_existing, df_date, on='URLs', how='left')

with pd.ExcelWriter(excel_file, engine="openpyxl", mode="a", if_sheet_exists="replace") as writer:
    df_updated.to_excel(writer, sheet_name=sheet_name, index=False)

In [ ]:
excel_file = 'urls.xlsx'
date_df = pd.read_excel(excel_file, sheet_name='2020 - 2024')

filtered_df = date_df[date_df['Date'] == "date not found"]
filtered_df

## Text Scraping

In [26]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

excel_file = 'urls.xlsx'
df = pd.read_excel(excel_file, sheet_name='2020 - 2024')

In [27]:
urls = df['URLs'].tolist()

text_data = []

headers = {
    'User-Agent': 'Mozilla/5.0 (Windows NT 10.0; Win64; x64) AppleWebKit/537.36 (KHTML, like Gecko) Chrome/91.0.4472.124 Safari/537.36'
}

for index, url in enumerate(urls):
    print(f"processing URL {index + 1}/{len(urls)}: {url}")
    try:
        response = requests.get(url, headers=headers)
        if response.status_code == 200:
            html = response.text

            soup = BeautifulSoup(html, 'html.parser')

            paragraphs = soup.find_all('p')
            extracted_text = '\n'.join([p.get_text() for p in paragraphs])

            text_data.append({'URLs': url, 'Text': extracted_text})
            print(extracted_text[:60])
        else:
            print(f"failed to retrieve {url}. status code: {response.status_code}")
            text_data.append({'URLs': url, 'Text': ''})
    except Exception as e:
        print(f"error fetching {url}: {str(e)}")
        text_data.append({'URLs': url, 'Text': ''}) 

text = pd.DataFrame(text_data)

processing URL 1/307: https://www.straitstimes.com/business/singapore-shophouse-sales-surge-prices-climb-as-wealthy-investors-return-knight-frank
Samuel Oh
SINGAPORE - Sales of shophouses in Singapore rose 
processing URL 2/307: https://www.straitstimes.com/business/singapore-s-largest-property-deal-of-2023-falls-apart-on-regulatory-snag
SINGAPORE - A blockbuster deal backed by a Chinese commoditi
processing URL 3/307: https://www.straitstimes.com/business/singapore-s-economic-growth-slows-quarter-on-quarter-on-weak-manufacturing
SINGAPORE - The economy grew in the first quarter of 2024 at
processing URL 4/307: https://www.straitstimes.com/business/singapore-factory-output-falls-92-in-march


KeyboardInterrupt: 

In [ ]:
df_updated = pd.merge(df, text, on='URLs', how='left')

def clean_string(s):
    if isinstance(s, str):
        return ''.join(c for c in s if ord(c) in range(32, 127))
    return s

df_updated['Text'] = df_updated['Text'].apply(clean_string)

df_updated.to_excel("texts.xlsx", index=False)


## Date Handling

In [37]:
import pandas as pd
import requests
from bs4 import BeautifulSoup

excel_file = 'texts.xlsx'
df = pd.read_excel(excel_file, sheet_name='2020 - 2024')
df['Date'] = df['Date'].str.slice(0, 10)

df.to_excel("dates.xlsx", index=False)